In [ ]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-08 15:37:34.390068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746733054.447670   30927 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746733054.472165   30927 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746733054.604785   30927 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746733054.604836   30927 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746733054.604837   30927 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [ ]:
# |Set scenario path
dataset_path = 'data/processed/construal'

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
num_parallel_envs = 15
total_envs = 15
device = "cpu" # cpu just because we're in a notebook





### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )

In [ ]:
### Support Functions ###

# |Function to extract filename from path
env_path2name = lambda path: path.split("/")[-1].split(".")[0]

# |Function to compute eucledian distance given two points
euclidean_distance = lambda point1, point2: math.sqrt(sum([(a - b) ** 2 for a, b in zip(point1, point2)]))



### Distance from ego ###
def get_construal_veh_distance_ego(env: GPUDriveConstrualEnv):
    '''
    Get the distance of each vehicle (or average) in the construal to the ego vehicle

    Args:
        env: The environment object

    Returns:
        dict: The average distance or a list of distances from the ego vehicle to each vehicle in the construal
    '''
    curr_data_batch = [env_path2name(env_path_) for env_path_ in env.data_batch]
    # |Populate dictionary with all relevant information
    info_dict = dict()
    for env_num, env_name in enumerate(curr_data_batch):
        info_dict[env_name] = dict()
        info_dict[env_name]['ego_index'] = torch.where(env.cont_agent_mask[env_num])[0].item()
        info_dict[env_name]['construal_indices'] = construal_indices[env_name]
    
    # |Get all vehicle distances
    all_pos = env.get_data_log_obj().pos_xy
    distance_dict = dict()

    for env_num, env_name in enumerate(curr_data_batch):
        distance_dict[env_name] = dict()
        all_distances = [euclidean_distance(all_pos[env_num][info_dict[env_name]['ego_index']][timestep_].cpu().numpy(),
                                            all_pos[env_num][veh_indx_][timestep_].cpu().numpy()) 
                            for veh_indx_ in range(len(all_pos[env_num]))
                            for timestep_ in range(len(all_pos[env_num][veh_indx_]))]
              
        if normalize:
            #2# |Normalize distances to [0,1] using min-max scaling 
            #2# |Multiplied by -1 as distance is a penalty term, greater values are associated with higher penalty
            all_distances = -1*( (np.array(all_distances) - np.min(all_distances)) / (np.max(all_distances) - np.min(all_distances)) )

        for curr_indices in info_dict[env_name]['construal_indices']:
            distance_dict[env_name][curr_indices] = [all_distances[i] for i in curr_indices]
            if average:
                if len(distance_dict[env_name][curr_indices]) > 0:
                    distance_dict[env_name][curr_indices] = sum(distance_dict[env_name][curr_indices])/len(distance_dict[env_name][curr_indices])
                else:
                    distance_dict[env_name][curr_indices] = 0
                    
    return distance_dict


In [ ]:
### Main Code ###
